### Задание 1

In [1]:
import pandas as pd

In [96]:
log = pd.read_csv(r'D:\Workspace\visit_log.csv', sep=';')


In [97]:
log.loc[log.traffic_source.str.contains('yandex|google'), 'source_type'] = 'organic'
log.loc[(log.traffic_source.str.contains('email')&log.region.str.contains('Russia')), 'source_type'] = 'ad'
log.loc[(log.traffic_source.str.contains('email')&~log.region.str.contains('Russia')), 'source_type'] = 'other'
log.loc[pd.isnull(log.source_type),'source_type'] = 'other'

In [98]:
log.head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,other
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,other
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,other
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,other
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,ad


### Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

    Прочитайте содержимое файла с датафрейм
    Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном


In [20]:
import re

In [10]:
urls = pd.read_csv(r'D:\Workspace\URLs.txt')

In [16]:
regex = r'/{1}\d{8}-'


In [65]:
urls.loc[urls.url.str.contains(regex)].head(10)

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
10,/middleeast/36131117-divizion-s-400-ne-zametil...
11,/economics/36065674-rossiyane-vozmutilis-minim...
12,/politics/36118047-otets-24-detey-potreboval-i...
13,/travel/36194479-v-sheremetevo-passazhiry-ustr...
14,/politics/35638742-nesolidno-mariya-zaharova-s...


### Задание 3


    Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [50]:
ratings = pd.read_csv(r'D:\Workspace\ml-latest-small/ratings.csv')

In [51]:
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')


In [52]:
grp_rtg = ratings.groupby('userId').agg({'timestamp': ['min', 'max'], 'rating': 'count'})

In [53]:
ids = grp_rtg[grp_rtg['rating']['count']>100]

In [54]:
ids['life_time'] = ids['timestamp']['max'] - ids['timestamp']['min']

<ipython-input-54-da3bc370eaf1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ids['life_time'] = ids['timestamp']['max'] - ids['timestamp']['min']


In [55]:
ids.head()

timestamp                     rating         life_time
                       min                 max  count                  
userId                                                                 
1      2000-07-30 18:08:19 2000-08-08 07:27:42    232   8 days 13:19:23
4      1999-12-13 09:47:08 2001-12-05 17:49:02    216 723 days 08:01:54
6      1996-10-17 11:51:49 1996-10-17 12:55:15    314   0 days 01:03:26
7      2005-01-25 06:43:36 2007-04-10 05:08:51    152 804 days 22:25:15
10     2016-02-12 18:25:53 2016-02-16 10:41:15    140   3 days 16:15:22

In [62]:
f"Среднее время жизни пользователей: {ids['life_time'].mean()}"

'Среднее время жизни пользователей: 517 days 17:22:47.734693888'

### Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

1. таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
2. аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [88]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)


In [89]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)


In [91]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)


In [92]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)


In [93]:
all_revenue = rzd.merge(auto, how='outer').merge(air, how='outer').fillna(0)

In [94]:
all_revenue

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [95]:
all_revenue.merge(client_base, how='left')

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4
